In [1]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader

import random
import os

from sklearn.metrics import f1_score,accuracy_score, confusion_matrix

from sklearn.model_selection import train_test_split

from model import BaseModel
from dataloader import BertDataset
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

/home/hyj/anaconda3/envs/bert/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda')

In [19]:
params = {
    'EPOCHS':3,
    'LEARNING_RATE':2e-5,
    'BATCH_SIZE':16,
    'SEED':45,
    'DATA_PATH': '../../make_data/preprocessed_data/good_bad_df.csv',
    'SAVE_PATH':'../Models/Klue_Bert_Sentence.pt',
    'transformer':"klue/bert-base",
    'max_length':512
}

In [ ]:
# params = {
#     'EPOCHS':3,
#     'LEARNING_RATE':2e-5,
#     'BATCH_SIZE':16,
#     'SEED':45,
#     'DATA_PATH': '../../make_data/preprocessed_data/good_bad_df.csv',
#     'SAVE_PATH':'../Models/Bert_Sentence.pt',
#     'transformer':"bert-base-multilingual-cased",
#     'max_length':512
# }

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(params['SEED']) # Seed 고정

In [4]:
data = pd.read_csv(params['DATA_PATH'],index_col=[0])

data.loc[data['ad_label']==1,'ad_label']=0
data.loc[data['ad_label']==2,'ad_label']=1

train_data, test_data = train_test_split(data,test_size=0.2,random_state=params['SEED'],shuffle=True)
valid_data, test_data = train_test_split(test_data,test_size=0.5,random_state=params['SEED'],shuffle=True)

train_data = train_data.sample(frac=1).reset_index(drop=True)
valid_data = valid_data.sample(frac=1).reset_index(drop=True)
test_data = test_data.sample(frac=1).reset_index(drop=True)

train_data.reset_index(drop=True,inplace=True)
valid_data.reset_index(drop=True,inplace=True)
test_data.reset_index(drop=True,inplace=True)

In [5]:
print(len(train_data))
print(len(train_data.drop_duplicates(['summary'])))

print(len(valid_data))
print(len(valid_data.drop_duplicates(['summary'])))

print(len(test_data))
print(len(test_data.drop_duplicates(['summary'])))

print(train_data['ad_label'].value_counts()/len(train_data))
print(valid_data['ad_label'].value_counts()/len(valid_data))
print(test_data['ad_label'].value_counts()/len(test_data))

6867
6867
858
858
859
859
0    0.696665
1    0.303335
Name: ad_label, dtype: float64
0    0.715618
1    0.284382
Name: ad_label, dtype: float64
0    0.704307
1    0.295693
Name: ad_label, dtype: float64


In [6]:
train_dataset = BertDataset(train_data,params)
train_dataloader = DataLoader(train_dataset,batch_size=params['BATCH_SIZE'])
valid_dataset = BertDataset(valid_data,params)
valid_dataloader = DataLoader(valid_dataset,batch_size=params['BATCH_SIZE'])
test_dataset = BertDataset(test_data,params)
test_dataloader = DataLoader(test_dataset,batch_size=params['BATCH_SIZE'])

In [7]:
def train(model, train_loader,valid_loader, params, device):
    
    model.to(device)

    criterion = nn.BCELoss()
    optimizer = optim.AdamW(model.parameters(),lr=params['LEARNING_RATE'])
    
    best_score = 0
    best_model = "None"
    for epoch_num in range(1,params["EPOCHS"]+1):

        model.train()
        
        train_loss = []
        for input_ids,masks,labels in tqdm(train_loader):
            
            train_input_ids = input_ids.to(device)
            train_masks = masks.to(device)
            train_labels = labels.to(device)
            
            optimizer.zero_grad()

            output = model(train_input_ids,train_masks)     
            output = output.reshape(-1)
            
            batch_loss = criterion(output.to(torch.float32), train_labels.to(torch.float32)) 
            train_loss.append(batch_loss.item())
            
            batch_loss.backward()
            optimizer.step()

        val_loss, val_acc, val_f1 = validation(model, criterion, valid_loader, device)
        print(f'Epoch [{epoch_num}], Train Loss : [{np.mean(train_loss) :.5f}] \
              Val Loss : [{np.mean(val_loss) :.5f}] Val Accuracy Score : [{val_acc:.5f}] Val F1 Score : [{val_f1:.5f}]')
        
        val_score = val_f1
        if best_score < val_score:
            best_model = model
            best_score = val_score
        
    return best_model                         

In [8]:
def competition_metric(true, pred):
    return accuracy_score(true,pred),f1_score(true, pred, average="macro")

def validation(model, criterion, test_loader, device):
    model.eval()

    val_loss = []
    model_preds = []
    true_labels = []  
    with torch.no_grad():
        for input_ids, masks, labels in tqdm(test_loader):
            valid_labels = labels.to(device)
            valid_input_ids = input_ids.to(device)
            valid_masks = masks.to(device)

            output = model(valid_input_ids, valid_masks)
            output = output.reshape(-1)
            
            batch_loss = criterion(output.to(torch.float32), valid_labels.to(torch.float32)) 
            val_loss.append(batch_loss.item())      
            
            output[output>0.5] = 1
            output[output<=0.5] = 0
            model_preds += output.detach().cpu().numpy().tolist()
            true_labels += valid_labels.detach().cpu().numpy().tolist()
        val_acc, val_f1 = competition_metric(true_labels, model_preds)
    return val_loss, val_acc, val_f1    

In [9]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    
    test_predict = []
    true_labels = []
    
    with torch.no_grad():
        for input_ids, masks, labels in tqdm(test_loader):
            test_labels = labels.to(device)
            test_input_ids= input_ids.to(device)
            test_masks = masks.to(device)

            output = model(test_input_ids, test_masks)  
            
            output[output>0.5] = 1
            output[output<=0.5] = 0
            test_predict += output.detach().cpu().numpy().tolist()
            true_labels += test_labels.detach().cpu().numpy().tolist()
    print('Done.')
    return test_predict

In [10]:
model = BaseModel(params)
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = params["LEARNING_RATE"])

infer_model = train(model, train_dataloader, valid_dataloader, params, device)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 54/54 [00:06<00:00,  8.41it/s]


Epoch [1], Train Loss : [0.14734]               Val Loss : [0.10521] Val Accuracy Score : [0.97203] Val F1 Score : [0.96484]


100%|██████████| 54/54 [00:06<00:00,  8.39it/s]


Epoch [2], Train Loss : [0.04003]               Val Loss : [0.06497] Val Accuracy Score : [0.97786] Val F1 Score : [0.97227]


100%|██████████| 54/54 [00:06<00:00,  8.37it/s]

Epoch [3], Train Loss : [0.02097]               Val Loss : [0.04595] Val Accuracy Score : [0.98951] Val F1 Score : [0.98716]


In [13]:
test_labels = test_data['ad_label']
test_preds = inference(infer_model,test_dataloader,device)

print(competition_metric(test_labels,test_preds))

print(confusion_matrix(test_labels,test_preds))

100%|██████████| 54/54 [00:06<00:00,  8.42it/s]

Done.
(0.9930151338766007, 0.9915958987203068)
[[603   2]
 [  4 250]]


In [14]:
labels = train_data['ad_label']
preds = inference(infer_model,train_dataloader,device)

print(competition_metric(labels,preds))

print(confusion_matrix(labels,preds))

100%|██████████| 430/430 [00:51<00:00,  8.34it/s]

Done.
(0.9982525120139799, 0.9979337985069912)
[[4776    8]
 [   4 2079]]


In [15]:
labels = valid_data['ad_label']
preds = inference(infer_model,valid_dataloader,device)

print(competition_metric(labels,preds))

print(confusion_matrix(labels,preds))

100%|██████████| 54/54 [00:06<00:00,  8.38it/s]


Done.
(0.9895104895104895, 0.987161561162143)
[[608   6]
 [  3 241]]


In [20]:
torch.save(infer_model.state_dict(),params['SAVE_PATH'])